In [4]:
## NEED TO SPIN UP THE WEAVIATE CONTAINER FIRST
## docker run -p 8080:8080 -p 50051:50051 semitechnologies/weaviate

import weaviate

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(10, 25))  # Values in seconds
)

/tmp/ipykernel_7438/2553797101.py:6: ResourceWarning: unclosed <socket.socket fd=73, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 58822), raddr=('127.0.0.1', 8080)>
  client = weaviate.connect_to_local(


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS


embeddings = OllamaEmbeddings()

path_to_pdf = 'Pregnancy and social welfare payments.pdf'
loader = PyPDFLoader(path_to_pdf)
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(pages)
retriever = FAISS.from_documents(docs, embeddings).as_retriever()

In [3]:
import weaviate
from weaviate.exceptions import RequestsConnectionError, SchemaValidationException, ObjectAlreadyExistsException

client = weaviate.Client("http://localhost:8080")

schema = {
    "classes": [
        {
            "class": "Person",
            "properties": [
                {"name": "name", "dataType": ["string"]},
                {"name": "age", "dataType": ["int"]},
            ],
        },
    ],
}

# Error handling for creating schema
try:
    client.schema.create(schema)
except (RequestsConnectionError, SchemaValidationException) as e:
    print(f"Failed to create schema: {e}")

# Data to be added
person = {"name": "John Doe", "age": 30}

# Error handling for adding data
try:
    client.data_object.create(person, "Person")
except (RequestsConnectionError, ObjectAlreadyExistsException) as e:
    print(f"Failed to add data object: {e}")


ImportError: cannot import name 'RequestsConnectionError' from 'weaviate.exceptions' (/root/miniconda3/envs/cit/lib/python3.10/site-packages/weaviate/exceptions.py)

In [ ]:
for page in pages:
    loader.get

AttributeError: 'PyPDFLoader' object has no attribute 'get'